<a href="https://colab.research.google.com/github/sarahbraia/dissertation_methodology_analysis_ITA/blob/main/dissertation_methodology_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

APPLICATION OF THE STUDY OF TIMES AND MOVEMENTS AND WRIGHT’S LAW

In [178]:
#Importando os dados do google drive
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
from IPython.display import display

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [179]:
# Reading files from Google Drive
base_path = '/content/drive/MyDrive/ITA -DISSERTAÇÃO/cabling_time_study_data.xlsx'
df = pd.read_excel(base_path)

# Preview of each dataset
df.head()

,OP,ETAPA,ID_OPERACAO,QTD,SETUP_HORA(S.H),EXECUCAO_HORA(E.H),TEMPO_TOTAL(T.T)
0,41810753,decapar os condutores,Decapar,1,00:02:34,00:02:28,00:05:02
1,41810753,corte do ajuste do tamanho das extremidades do...,Cortar,1,00:02:48,00:00:40,00:03:28
2,41810753,crimpagem,Crimpar,1,00:01:11,00:00:00,00:01:11
3,41810753,ABERTURA DE ANDON - CONECTOR PAGO ERRADO,Abrir chamado,1,00:09:00,00:04:29,00:13:29
4,41810753,Analise do Andon pelo hardwer,Abrir chamado,1,00:07:00,00:00:00,00:07:00


### Application of Micromovement Study in Cable Assembly

In [180]:
# Convert time columns from string to datetime format
df['SETUP_HORA(S.H)'] = pd.to_datetime(df['SETUP_HORA(S.H)'], format='%H:%M:%S', errors='coerce')
df['EXECUCAO_HORA(E.H)'] = pd.to_datetime(df['EXECUCAO_HORA(E.H)'], format='%H:%M:%S', errors='coerce')
df['TEMPO_TOTAL(T.T)'] = pd.to_datetime(df['TEMPO_TOTAL(T.T)'], format='%H:%M:%S', errors='coerce')

# Replace missing values with 00:00:00
df['SETUP_HORA(S.H)'] = df['SETUP_HORA(S.H)'].fillna(pd.to_datetime('00:00:00'))
df['EXECUCAO_HORA(E.H)'] = df['EXECUCAO_HORA(E.H)'].fillna(pd.to_datetime('00:00:00'))
df['TEMPO_TOTAL(T.T)'] = df['TEMPO_TOTAL(T.T)'].fillna(pd.to_datetime('00:00:00'))

# Function to convert time to seconds
def time_to_seconds(t):
    return t.hour * 3600 + t.minute * 60 + t.second

# Apply the function to extract seconds from each time column
df['SETUP_SECONDS'] = df['SETUP_HORA(S.H)'].dt.time.apply(time_to_seconds)
df['EXECUTION_SECONDS'] = df['EXECUCAO_HORA(E.H)'].dt.time.apply(time_to_seconds)
df['TOTAL_SECONDS'] = df['TEMPO_TOTAL(T.T)'].dt.time.apply(time_to_seconds)

# Preview the result
df[['ID_OPERACAO', 'SETUP_SECONDS', 'EXECUTION_SECONDS', 'TOTAL_SECONDS']].head(15)

,ID_OPERACAO,SETUP_SECONDS,EXECUTION_SECONDS,TOTAL_SECONDS
0,Decapar,154,148,302
1,Cortar,168,40,208
2,Crimpar,71,0,71
3,Abrir chamado,540,269,809
4,Abrir chamado,420,0,420
5,Preparar bancada,116,41,157
6,Organizar os fios,120,25,145
7,Crimpar,0,35,35
8,Medir tamanho do cabo,0,19,19
9,Cortar,117,15,132


In [181]:
# Dictionary for ID and English operation name mapping
operation_mapping = {
    'Abrir chamado': (1, 'Open service request'),
    'Fabricar adesivo': (2, 'Produce adhesive'),
    'Teste de continuidade': (3, 'Continuity test'),
    'Soprador térmico': (4, 'Apply heat gun'),
    'Montar Conector': (5, 'Assemble connector'),
    'Inserir': (6, 'Insert'),
    'Cortar': (7, 'Cut'),
    'Crimpar': (8, 'Crimp'),
    'Decapar': (9, 'Strip wire'),
    'Organizar os fios': (10, 'Organize wires'),
    'Estanhar': (11, 'Tin wires'),
    'Soldar': (12, 'Solder'),
    'Limpar excesso de solda': (13, 'Remove excess solder'),
    'Preparar bancada': (14, 'Prepare workstation'),
    'Marcar': (15, 'Mark'),
    'Medir tamanho do cabo': (16, 'Measure cable length')
}

# Create the new columns using the mapping
df['ID'] = df['ID_OPERACAO'].map(lambda x: operation_mapping.get(x, (None, None))[0])
df['Operation_ID_(EN)'] = df['ID_OPERACAO'].map(lambda x: operation_mapping.get(x, (None, None))[1])

# Rename the column
df.rename(columns={'ID_OPERACAO': 'ID_OPERAÇÃO_(PT)'}, inplace=True)

# Preview the result
df[['ID', 'ID_OPERAÇÃO_(PT)', 'Operation_ID_(EN)', 'TOTAL_SECONDS']].head(15)

,ID,ID_OPERAÇÃO_(PT),Operation_ID_(EN),TOTAL_SECONDS
0,9,Decapar,Strip wire,302
1,7,Cortar,Cut,208
2,8,Crimpar,Crimp,71
3,1,Abrir chamado,Open service request,809
4,1,Abrir chamado,Open service request,420
5,14,Preparar bancada,Prepare workstation,157
6,10,Organizar os fios,Organize wires,145
7,8,Crimpar,Crimp,35
8,16,Medir tamanho do cabo,Measure cable length,19
9,7,Cortar,Cut,132


#### Refined cabling operation

In [182]:
# Group by ID and Operation_ID_(EN) and count occurrences
distinct_counts = (
    df.groupby(['ID', 'Operation_ID_(EN)'])
    .size()
    .reset_index(name='COUNT')
    .sort_values(by='ID')
)

# Display the result
distinct_counts


,ID,Operation_ID_(EN),COUNT
0,1,Open service request,2
1,2,Produce adhesive,29
2,3,Continuity test,25
3,4,Apply heat gun,56
4,5,Assemble connector,25
5,6,Insert,68
6,7,Cut,55
7,8,Crimp,30
8,9,Strip wire,109
9,10,Organize wires,7


#### Number of cycles to be timed

##### Calculate the range (R) per refined operation, the mean (X) and ratio (R/X)

In [183]:
# Group by operation and calculate the highest (H) and lowest (L) times
analysis_df = df.groupby('Operation_ID_(EN)').agg(
    H=('TOTAL_SECONDS', 'max'),
    L=('TOTAL_SECONDS', 'min')
).reset_index()

# Calculate range (R = H - L)
analysis_df['R'] = analysis_df['H'] - analysis_df['L']

# Calculate approximate mean (X̄ ≈ (H + L) / 2)
analysis_df['X̄'] = (analysis_df['H'] + analysis_df['L']) / 2

# Calculate relative variability ratio (R / X̄)
analysis_df['R/X̄'] = (analysis_df['R'] / analysis_df['X̄']).round(2)

# Calculate IQR-based range instead of absolute max-min
def iqr_range(series):
    q3 = series.quantile(0.75)
    q1 = series.quantile(0.25)
    return q3 - q1, (q3 + q1) / 2  # Also returns adjusted X̄

# Apply per operation
iqr_df = df.groupby('Operation_ID_(EN)')['TOTAL_SECONDS'].apply(iqr_range).apply(pd.Series)
iqr_df.columns = ['R_IQR', 'X̄_IQR']
iqr_df['R/X̄'] = (iqr_df['R_IQR'] / iqr_df['X̄_IQR']).round(2)

# Display final result
display(analysis_df)


,Operation_ID_(EN),H,L,R,X̄,R/X̄
0,Apply heat gun,121,10,111,65.5,1.69
1,Assemble connector,373,53,320,213.0,1.50
2,Continuity test,640,8,632,324.0,1.95
3,Crimp,149,10,139,79.5,1.75
4,Cut,208,7,201,107.5,1.87
5,Insert,135,12,123,73.5,1.67
6,Mark,138,6,132,72.0,1.83
7,Measure cable length,77,10,67,43.5,1.54
8,Open service request,809,420,389,614.5,0.63
9,Organize wires,149,18,131,83.5,1.57


In [184]:
# Reading files from Google Drive
table_relative_error = '/content/drive/MyDrive/ITA -DISSERTAÇÃO/table_relative_error.xlsx'
table_relative_error_df = pd.read_excel(table_relative_error)
# Preview of each dataset
table_relative_error_df.head()

,R/X,relative_error_of_5%,relative_error_of_10%
0,0.10,3,2
1,0.12,4,2
2,0.14,6,3
3,0.16,8,4
4,0.18,10,6


In [185]:
# Calculate IQR-based range and adjusted mean (X̄) per operation
def iqr_range(series):
    q3 = series.quantile(0.75)
    q1 = series.quantile(0.25)
    return q3 - q1, (q3 + q1) / 2

# Apply IQR calculation
iqr_df = df.groupby('Operation_ID_(EN)')['TOTAL_SECONDS'].apply(iqr_range).apply(pd.Series)
iqr_df.columns = ['R_IQR', 'X̄_IQR']

# Calculate R/X̄ and round to two decimals
iqr_df['R/X'] = (iqr_df['R_IQR'] / iqr_df['X̄_IQR']).round(2)

# Function to find closest R/X and return required N for 10% error
def find_nearest_n(ratio):
    idx = (table_relative_error_df['R/X'] - ratio).abs().idxmin()
    return table_relative_error_df.loc[idx, 'relative_error_of_10%']

# Apply function to determine required number of observations
iqr_df['N_required_10%'] = iqr_df['R/X'].apply(find_nearest_n)

# Get the number of observations per operation
count_df = df.groupby('Operation_ID_(EN)').size().reset_index(name='COUNT')

# Merge with IQR table
final_df = iqr_df.merge(count_df, on='Operation_ID_(EN)')

# Display final result
from IPython.display import display
display(final_df)


,Operation_ID_(EN),R_IQR,X̄_IQR,R/X,N_required_10%,COUNT
0,Apply heat gun,37.50,40.500,0.93,149,56
1,Assemble connector,111.00,160.500,0.69,78,25
2,Continuity test,51.00,78.500,0.65,69,25
3,Crimp,37.00,51.250,0.72,88,30
4,Cut,15.00,22.000,0.68,78,55
5,Insert,43.25,43.625,0.99,162,68
6,Mark,21.00,25.500,0.82,113,57
7,Measure cable length,32.50,39.250,0.83,113,27
8,Open service request,194.50,614.500,0.32,17,2
9,Organize wires,60.00,71.000,0.85,119,7


#### Calculation of normal time.

In [188]:
# Group by operation and calculate the average time
summary_df = df.groupby('Operation_ID_(EN)')['TOTAL_SECONDS'].mean().reset_index()
summary_df.rename(columns={'TOTAL_SECONDS': 'selected_time'}, inplace=True)

# Calculate Normal Time (115% pace factor) and Standard Time (Normal + 5% tolerance)
summary_df['Normal_Time'] = (summary_df['selected_time'] * 1.15).round(2)
summary_df['Standard_Time'] = (summary_df['Normal_Time'] * 1.05).round(2)

# Display final result
from IPython.display import display
display(summary_df)

,Operation_ID_(EN),selected_time,Normal_Time,Standard_Time
0,Apply heat gun,43.357143,49.86,52.35
1,Assemble connector,160.600000,184.69,193.92
2,Continuity test,124.520000,143.20,150.36
3,Crimp,59.033333,67.89,71.28
4,Cut,34.127273,39.25,41.21
5,Insert,46.911765,53.95,56.65
6,Mark,30.421053,34.98,36.73
7,Measure cable length,39.481481,45.40,47.67
8,Open service request,614.500000,706.68,742.01
9,Organize wires,72.142857,82.96,87.11
